### Question 3 - [10 marks]

##### Use dataset available on http://users.stat.ufl.edu/~winner/data/nfl2008_fga.csv

##### a.	Use LDA to classify the dataset into few classes so that at least 90% of information of dataset is explained through new classification. (Hint: model the variable “qtr” to variables “togo”, “kicker”, and “ydline”). How many LDs do you choose? Explain the reason.                                                           [5]                      
##### b.	Apply PCA, and identify the important principle components involving at least 90% of dataset variation. Explain your decision strategy?  Plot principle components versus their variance.  [5]                                       


In [1]:
#Importing all the necessary libraries for the project.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,classification_report,confusion_matrix

In [2]:
#The above code is reading the data from the url and storing it in a variable called data.
url = 'http://users.stat.ufl.edu/~winner/data/nfl2008_fga.csv'
data = pd.read_csv(url)
data.head()

,GameDate,AwayTeam,HomeTeam,qtr,min,sec,kickteam,def,down,togo,...,distance,homekick,kickdiff,timerem,offscore,defscore,season,GOOD,Missed,Blocked
0,20081130,IND,CLE,1,47,2,IND,CLE,4.0,11.0,...,30,0,-3,2822,0,3,2008,1,0,0
1,20081005,IND,HOU,1,54,47,IND,HOU,4.0,3.0,...,46,0,0,3287,0,0,2008,1,0,0
2,20081228,TEN,IND,1,45,20,IND,TEN,4.0,3.0,...,28,1,7,2720,7,0,2008,1,0,0
3,20081012,BAL,IND,1,45,42,IND,BAL,4.0,1.0,...,37,1,14,2742,14,0,2008,1,0,0
4,20080907,CHI,IND,1,50,56,IND,CHI,4.0,21.0,...,39,1,0,3056,0,0,2008,1,0,0


In [3]:
#Check Nulls
data.isnull().sum()

GameDate    0
AwayTeam    0
HomeTeam    0
qtr         0
min         0
sec         0
kickteam    0
def         0
down        2
togo        2
kicker      0
ydline      0
name        0
distance    0
homekick    0
kickdiff    0
timerem     0
offscore    0
defscore    0
season      0
GOOD        0
Missed      0
Blocked     0
dtype: int64

In [4]:
#Check Stats
data.describe()

,GameDate,qtr,min,sec,down,togo,kicker,ydline,distance,homekick,kickdiff,timerem,offscore,defscore,season,GOOD,Missed,Blocked
count,1.039000e+03,1039.000000,1039.000000,1039.000000,1037.000000,1037.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.0,1039.000000,1039.000000,1039.000000
mean,2.008140e+07,2.497594,27.968239,26.551492,3.818708,6.738669,19.537055,18.768046,36.666025,0.494706,0.382098,1704.645813,9.949952,9.567854,2008.0,0.866218,0.133782,0.021174
std,1.702338e+03,1.108621,16.716783,18.107977,0.635134,4.155772,11.303244,10.187505,9.847836,0.500213,9.675133,1003.771275,9.434869,8.365389,0.0,0.340582,0.340582,0.144034
min,2.008090e+07,1.000000,-15.000000,0.000000,1.000000,1.000000,1.000000,1.000000,18.000000,0.000000,-41.000000,-887.000000,0.000000,0.000000,2008.0,0.000000,0.000000,0.000000
25%,2.008100e+07,2.000000,14.000000,9.500000,4.000000,4.000000,9.000000,10.000000,28.000000,0.000000,-6.000000,895.500000,0.000000,3.000000,2008.0,1.000000,0.000000,0.000000
50%,2.008111e+07,2.000000,30.000000,25.000000,4.000000,6.000000,20.000000,19.000000,37.000000,0.000000,0.000000,1808.000000,7.000000,7.000000,2008.0,1.000000,0.000000,0.000000
75%,2.008120e+07,4.000000,42.000000,43.000000,4.000000,9.000000,30.000000,26.000000,44.000000,1.000000,6.000000,2556.500000,16.000000,14.000000,2008.0,1.000000,0.000000,0.000000
max,2.009020e+07,5.000000,58.000000,59.000000,4.000000,25.000000,37.000000,86.000000,76.000000,1.000000,44.000000,3507.000000,48.000000,44.000000,2008.0,1.000000,1.000000,1.000000


In [5]:
#Printing the unique values of the column 'qtr' in the dataframe data.
print('Class labels:', np.unique(data['qtr']))

Class labels: [1 2 3 4 5]


In [6]:
#Filled Nulls with '0'
data = data.fillna(0)
data.isnull().sum().count

<bound method Series.count of GameDate    0
AwayTeam    0
HomeTeam    0
qtr         0
min         0
sec         0
kickteam    0
def         0
down        0
togo        0
kicker      0
ydline      0
name        0
distance    0
homekick    0
kickdiff    0
timerem     0
offscore    0
defscore    0
season      0
GOOD        0
Missed      0
Blocked     0
dtype: int64>

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039 entries, 0 to 1038
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   GameDate  1039 non-null   int64  
 1   AwayTeam  1039 non-null   object 
 2   HomeTeam  1039 non-null   object 
 3   qtr       1039 non-null   int64  
 4   min       1039 non-null   int64  
 5   sec       1039 non-null   int64  
 6   kickteam  1039 non-null   object 
 7   def       1039 non-null   object 
 8   down      1039 non-null   float64
 9   togo      1039 non-null   float64
 10  kicker    1039 non-null   int64  
 11  ydline    1039 non-null   int64  
 12  name      1039 non-null   object 
 13  distance  1039 non-null   int64  
 14  homekick  1039 non-null   int64  
 15  kickdiff  1039 non-null   int64  
 16  timerem   1039 non-null   int64  
 17  offscore  1039 non-null   int64  
 18  defscore  1039 non-null   int64  
 19  season    1039 non-null   int64  
 20  GOOD      1039 non-null   int6

In [8]:

#X= data[['togo','kicker', 'ydline']]
X = data[['min',	'sec'	,'down'	,'togo'	,'kicker'	,'ydline'	,'distance'	,'homekick'	,'kickdiff'	,'timerem',	'offscore'	,'defscore'	,'season'	,'GOOD',	'Missed'	,'Blocked']]
y= data['qtr']

In [9]:
X.shape

(1039, 16)

In [10]:
y.unique()

array([1, 2, 3, 4, 5], dtype=int64)

In [11]:
#The above code is splitting the data into training and testing data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)

In [12]:
#The above code is standardizing the data.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
#Applying Decision Tree on Original Dataset

acc = [] ### Blank vector
for i in range(1000):
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=30)# split dataset
  model_dt=DecisionTreeClassifier() # model
  dt = model_dt.fit(X_train,y_train) # fit the model
  YPred = dt.predict(X_test) # predict
  a = accuracy_score(y_test,YPred) # compute accuracy
  acc.append(a) # append accuracy
print("Accuracy using 1000 MC run:", np.mean(acc))  # average of accuracy to stablize the result


Accuracy using 1000 MC run: 0.9967948717948721


#### LDA
- Creates linear combination of the original features
- Number of such transformations are 1 less than the number of classes (the classes are well seperated)
- Very similar to PCA, LDA is supervised

```math
ld1 = 0.5*x1 + 0.2*x2 + 0.3*x3
```

In [14]:
#Initializing LDA
lda_model = LinearDiscriminantAnalysis(n_components=1)
X_lda = lda_model.fit_transform(X, y)

In [15]:
#Checking Variance Ratio
lda_var_ratios = lda_model.explained_variance_ratio_

In [16]:
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [17]:
#Selecting n components with 95% explained variance
select_n_components(lda_var_ratios, 0.95)

1

In [18]:
X_lda.shape

(1039, 1)

In [19]:
LDA=pd.DataFrame(X_lda)
LDA.columns=['lda1']

In [20]:
a=lda_model.explained_variance_ratio_
a

array([0.99244456])

In [21]:
# plt.plot(np.cumsum(a))
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance')

In [22]:
#Training model after LDA

from sklearn.model_selection import train_test_split 
acc = [] ### Blank vector
for i in range(1000):
  X_train,X_test,y_train,y_test=train_test_split(X_lda,y,test_size=0.3,random_state=30)# split dataset
  model_dt=DecisionTreeClassifier() # model
  dt = model_dt.fit(X_train,y_train) # fit the model
  YPred = dt.predict(X_test) # predict
  a = accuracy_score(y_test,YPred) # compute accuracy
  acc.append(a) # append accuracy
print("Accuracy using 1000 MC run:", np.mean(acc))  # average of accuracy to stablize the result

Accuracy using 1000 MC run: 0.9775641025641023


In [23]:
X.shape

(1039, 16)

In [24]:
#PCA

# create the PCA instance
pca = PCA(n_components=1)
x = pca.fit_transform(X)
# x.shape
# fit on data
# pca.fit(X)
# access values and vectors
PCAs=pca.components_
df=pd.DataFrame(PCAs)
df_pca=df.T  # transpose the dataframe 
df_pca.columns=['pca1']
df_pca.head()

,pca1
0,-0.016648
1,-0.000924
2,-0.000070
3,0.000101
4,0.000372


In [25]:
#Variance explained by each component
print(pca.explained_variance_)
pca.n_components_

[1007908.25197757]


1

In [26]:
ratio_var=pca.explained_variance_/sum(pca.explained_variance_)

In [27]:
df_new=np.dot(X, df_pca)

In [28]:
#Training model after PCA

acc = [] ### Blank vector
for i in range(1000):
  X_train,X_test,y_train,y_test=train_test_split(df_new,y,test_size=0.3,random_state=30)# split dataset
  model_lr=LogisticRegression() # model
  lr = model_lr.fit(X_train,y_train) # fit the model
  YPred = lr.predict(X_test) # predict
  a = accuracy_score(y_test,YPred) # compute accuracy
  acc.append(a) # append accuracy
print("Logistic Regression Accuracy using 1000 MC run:", np.mean(acc))  # average of accuracy to stablize the result


C:\Users\Swaraj\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Swaraj\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

Logistic Regression Accuracy using 1000 MC run: 0.9967948717948721


In [29]:

acc = [] ### Blank vector
for i in range(1000):
  X_train,X_test,y_train,y_test=train_test_split(df_new,y,test_size=0.3,random_state=30)# split dataset
  model_dt=DecisionTreeClassifier() # model
  dt = model_dt.fit(X_train,y_train) # fit the model
  YPred = dt.predict(X_test) # predict
  a = accuracy_score(y_test,YPred) # compute accuracy
  acc.append(a) # append accuracy
print("Decision Tree Accuracy using 1000 MC run:", np.mean(acc))  # average of accuracy to stablize the result


Decision Tree Accuracy using 1000 MC run: 0.9967948717948721


In [30]:

acc = [] ### Blank vector
for i in range(1000):
  X_train,X_test,y_train,y_test=train_test_split(df_new,y,test_size=0.3,random_state=30)# split dataset
  model_nb=GaussianNB() # model
  nb = model_nb.fit(X_train,y_train) # fit the model
  YPred = nb.predict(X_test) # predict
  a = accuracy_score(y_test,YPred) # compute accuracy
  acc.append(a) # append accuracy
print("Gaussian Naive Bayes Accuracy using 1000 MC run:", np.mean(acc))  # average of accuracy to stablize the result


Gaussian Naive Bayes Accuracy using 1000 MC run: 0.9647435897435896


In [31]:

acc = [] ### Blank vector
for i in range(1000):
  X_train,X_test,y_train,y_test=train_test_split(df_new,y,test_size=0.3,random_state=30)# split dataset
  model_rf=GaussianNB() # model
  rf = model_rf.fit(X_train,y_train) # fit the model
  YPred = rf.predict(X_test) # predict
  a = accuracy_score(y_test,YPred) # compute accuracy
  acc.append(a) # append accuracy
print("Random Forest Accuracy using 1000 MC run:", np.mean(acc))  # average of accuracy to stablize the result


Random Forest Accuracy using 1000 MC run: 0.9647435897435896


## Insights
On testing the NFL dataset on 3 ML Algorithms both on LDA and PCA

- **Decision Tree** : Accuracy - 0.9967948717948721 (SUPER FAST) Less Time/Space Complexity
- **Naive Bayes** : Accuracy - 0.9647435897435896
- **Logistic Regression** : Accuracy - 0.9967948717948721 (Takes a lot of time to model) High Time/Space Complexity

| Models              | Accuracy [Original Dataset] | Accuracy [After LDA] | Accuracy [After PCA] |
|---------------------|-----------------------------|----------------------|----------------------|
| Decision Tree       | 0.9982                      | 0.9741               | 0.9967               |
| Logistic Regression | 0.9871                      | 0.9748               | 0.9967               |
| Naive Bayes         | 0.9100                      | 0.9749               | 0.9647               |
| Random Forest       | 0.9700                      | 0.9749               | 0.9647               |


For both, LDA and PCA the best **n_components_** was **1** for **>=0.95** explained variance.
Increasing number of **n_components** speeds up the modelling time but takes a hit on the accuracy of the trained model.